## Homework 4

**Submitted by : Tanvi Arora**   
**Section     : DS 7337 Natural Language Processing - 401**

<a id="top"></a>
### Contents

* <a href="#functions">Function Definitions</a>
* <a href="#corpus">Corpus and Data prep</a>
* <a href="#patternpos">1 - POS Tagger in Pattern library</a>
* <a href="#patternposlong">1.a - Longest sentence using pattern POS tagger</a>
* <a href="#patternposshort">1.b - Shortest sentence using pattern POS tagger</a>
* <a href="#spacypos">2 -POS Tagger in spaCy library</a>
* <a href="#spacyposlong">Longest sentence using spaCy POS tagger</a>
* <a href="#spacyposshort">Shortest sentence using spaCy POS tagger</a>
* <a href="#comppatternvsspacy">2.a - Compare pattern POS vs spaCy POS</a>
* <a href="#exppatternvsspacy">2.b - Explain pattern POS vs spaCy POS</a>
* <a href="#news">3 - News article</a>
* <a href="#newsmanual">3.a - Manually POS POS Tag using Penn Tagset</a>
* <a href="#newscomppatternvsspacy">3.b - Compare manual POS vs Pattern POS vs spaCy POS</a>
* <a href="#newsexppatternvsspacy">3.c - Explain manual POS vs Pattern POS vs spaCy POS</a>
* <a href="#addtagers">Additional Taggers : ClassiferBasedPOSTagger</a>

In [5]:
import platform
print(platform.platform())

import os
print ("environment",os.environ['CONDA_DEFAULT_ENV'])

import sys
print("Python",sys.version)

import nltk
from nltk.tokenize import regexp_tokenize
print("nltk",nltk.__version__)

## corpus referred
from nltk.corpus import brown

from nltk.tag.util import tuple2str

## for visualizations
import matplotlib.pyplot as plt
import pandas as pd

## pattern library
from pattern.en import tag
from pattern.text import UNIVERSAL

## spaCy library
import spacy
sp_nlp=spacy.load('en')
from spacy import displacy

pd.set_option('display.max_rows',10)
import re

## ignore/suppress warnings
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)

# To display plots inside the iPython Notebook itself
%matplotlib inline

Darwin-18.5.0-x86_64-i386-64bit
environment base
Python 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
nltk 3.4


**Approach followed**  
1. create tagged words using your POS tagger  
2. validate POS tag  :
    -- chk length , if length is not equal then strip punctuations  
    -- if length still does nto match, then output not matching POS tag  
    -- if length matches compare the 2 POS tags.  
    -- if len(match) <> len of inputs then POS tags are not matching. output string tuple  
    


<a id="functions"></a>
<a href="#top">Back to Top</a>

### Function Definitions

In [6]:
## Pattern universal tagset had same number of tags but different names ( used 2 char names)
## Brown corpus uses more than 3 char names for universal tagset
## Below dictionary is to convert pattern tags to match brown corpus tag names
pt_universal_tags={"NN":"NOUN",
               "VB":"VERB",
                "JJ":"ADJ",
                "RB":"ADV",
                "PR" : "PRON",
                "DT" :"DET",
                "PP" :"PREP",
                "NO":"NUM" ,
                "CJ":"CONJ" ,
                "UH" :"INTJ",
                "PT":"PRT" ,
                "." : ".",
                "X"  :"X"}

## spaCy universal tagset uses Google Universal dataset which is Universal dataset with a few additions
## Below dictionary is to convert spaCy tags to match brown corpus tag names. Names missing in brown corpus 
## will be added with same names as in spaCy , they would not match 
sp_universal_tags={"NOUN":"NOUN",
               "VERB":"VERB",
                "ADJ":"ADJ",
                "ADP":"ADP",
                "ADV":"ADV",
                "AUX":"AUX",
                "PRON" : "PRON",
                "DET" :"DET",
                "SCONJ":"SCONJ",
                "CONJ":"CONJ" ,
                "CCONJ":"CCONJ",
                "INTJ" :"INTJ",
                "PART":"PRT" ,
                "PUNCT"  :".",
                "X"  :"X",
                "NUM":"NUM",
                "PROPN":"PROPN",
                "SYM":"."}

## Below function will convert tokens to a sentence.
## brown corpus tagged_sents returns a list of sentence-tokens
def tokens2sent(tokens):
    return ' '.join(tokens)

## removes punctions from the list of tagged words
def remove_punct(tagged_word):
    return [(tag[0],tag[1]) for tag in tagged_word if tag[0].isalpha()]

## This function prints the tagged words in the form of list ( word,tag) as word/tag and joins them with a space
## to form a sentene type output
def print_tup2str(tagged_word):
    tagged_str=[tuple2str(t1) for t1 in tagged_word]
    return ' '.join(tagged_str)

## Display output of comparision as a message with the input and output
def print_output(verdict,golden_tag,pos_tag,pos_tag_name):
    if(verdict):
        print("POS tag is correct and matches the golden tag")
    else:
        print("POS tag may not be correct as it does not matches the golden tag")
    print("golden tag ---:")
    print(print_tup2str(golden_tag))
    print("POS tag ---", pos_tag_name, ":")
    print(print_tup2str(pos_tag))

## Compare function that takes a golden key set and pos tagger keyset
def compare_pos(golden_tag,pos_tag,pos_tag_name,display):
    #if len(golden_tag)!=len(pos_tag):
        #golden_tag=remove_punct(golden_tag)
        #pos_tag=remove_punct(pos_tag)
    if len(golden_tag)!=len(pos_tag):
        verdict=False
    else:
        match=[i for i,j in zip(golden_tag,pos_tag) if i==j]
        if len(match)==len(golden_tag):
            verdict=True
        else:
            verdict=False
    if(display):
        print_output(verdict,golden_tag,pos_tag,pos_tag_name)
    return verdict

## apply POS Tagger from Pattern using UNIVERSAL tagset
## in case the tagset has any combination tags, strip anything after first hyphen
def pos_pattern(sentence):
    pattern_tag=tag(sentence,tagset=UNIVERSAL)
    return [(t[0],pt_universal_tags[t[1].split("-",1)[0]]) for t in pattern_tag]

## apply POS tagger from spaCy using UNIVERSAL tagset
def pos_spacy(sentence):
    return [(token.text,sp_universal_tags[token.pos_]) for token in sp_nlp(sentence)]

## apply POS Tagger from Pattern using default/ Penn Tag Tagset
## in case the tagset has any combination tags, strip anything after first hyphen
def pos_pattern_penn(sentence):
    pattern_penn_tag=tag(sentence)
    return [(t[0],t[1].split("-",1)[0]) for t in pattern_penn_tag]


## apply POS Tagger from spaCy using default/ Penn Tag Tagset
def pos_spacy_penn(sentence):
    return [(token.text,token.tag_) for token in sp_nlp(sentence)]

## returns pattern POS tags for the sentence at a given index along with its comparision with golden tagset
def pos_pattern_disp_byindex(ind,golden_tag_lst,input_df,pos_tag_name):
    pattern_pos_ind=pos_pattern(input_df.sentence.iloc[ind])
    brown_golden_ind=golden_tag_lst[ind]
    print("original sentence --- :")
    print(input_df.sentence.iloc[ind])
    compare_pos(brown_golden_ind,pattern_pos_ind,pos_tag_name,True)

## returns spaCy POS tags for the sentence at a given index along with its comparision with golden tagset
def pos_spacy_disp_byindex(ind,golden_tag_lst,input_df,pos_tag_name):
    spacy_pos_ind=pos_spacy(input_df.sentence.iloc[ind])
    brown_golden_ind=golden_tag_lst[ind]
    print("original sentence --- :")
    print(input_df.sentence.iloc[ind])
    compare_pos(brown_golden_ind,spacy_pos_ind,pos_tag_name,True)
    
## returns pattern POS tags for the sentence at a given index along with its comparision with golden tagset
def pos_pattern_penn_disp_byindex(ind,golden_tag_lst,input_df,pos_tag_name):
    pattern_penn_pos_ind=pos_pattern_penn(input_df.sentence.iloc[ind])
    brown_golden_ind=golden_tag_lst[ind]
    print("original sentence --- :")
    print(input_df.sentence.iloc[ind])
    compare_pos(brown_golden_ind,pattern_penn_pos_ind,pos_tag_name,True)

## returns spaCy POS tags for the sentence at a given index along with its comparision with golden tagset
def pos_spacy_penn_disp_byindex(ind,golden_tag_lst,input_df,pos_tag_name):
    spacy_penn_pos_ind=pos_spacy_penn(input_df.sentence.iloc[ind])
    brown_golden_ind=golden_tag_lst[ind]
    print("original sentence --- :")
    print(input_df.sentence.iloc[ind])
    compare_pos(brown_golden_ind,spacy_penn_pos_ind,pos_tag_name,True)

<a id="corpus"></a>
<a href="#top">Back to Top</a>
### About the Corpus under study - Brown Corpus

The Brown University Standard Corpus of Present-Day American English (or just Brown Corpus) was compiled in the 1960s by Henry Kučera and W. Nelson Francis at Brown University, Providence, Rhode Island as a general corpus (text collection) in the field of corpus linguistics. It contains 500 samples of English-language text, totaling roughly one million words, compiled from works published in the United States in 1961.

The tagged Brown Corpus used a selection of about 80 parts of speech, as well as special indicators for compound forms, contractions, foreign words and a few other phenomena, and formed the basis for many later corpora such as the Lancaster-Oslo-Bergen Corpus. Some versions of the tagged Brown corpus contain combined tags. To perform auto compare of the POS tags returned by other Taggers, we have used the Universal tagset which is a shorter tagset of around 12 tags

### UNIVERSAL POS tags

Alphabetical listing  

ADJ: adjective  
ADP: adposition  
ADV: adverb  
AUX: auxiliary  
CCONJ: coordinating conjunction  
DET: determiner  
INTJ: interjection  
NOUN: noun  
NUM: numeral  
PART: particle  
PRON: pronoun  
PROPN: proper noun  
PUNCT: punctuation  
SCONJ: subordinating conjunction  
SYM: symbol  
VERB: verb  
X: other  

In [8]:
# Consider Brown corpus tagged words as the golden tagset for comparision
brown_goldentag=brown.tagged_sents(tagset='universal')

In [9]:
# Create a dataframe using brown corpus sentences , with length of each sentence( token list)
# which will be further used for comparisions
sent_dyn=[(c,len(value),value) for c,value in enumerate(brown.sents(),0)]

In [10]:
# Brown corpus has 57340 sentences in total from all categories
len(sent_dyn)

57340

In [11]:
sent_df=pd.DataFrame(sent_dyn)
sent_df.columns=["position","length","tokens"]
sent_df["sentence"]=[tokens2sent(t) for t in sent_df["tokens"]]
sent_df.head()

,position,length,tokens,sentence
0,0,25,"[The, Fulton, County, Grand, Jury, said, Frida...",The Fulton County Grand Jury said Friday an in...
1,1,43,"[The, jury, further, said, in, term-end, prese...",The jury further said in term-end presentments...
2,2,35,"[The, September-October, term, jury, had, been...",The September-October term jury had been charg...
3,3,37,"[``, Only, a, relative, handful, of, such, rep...",`` Only a relative handful of such reports was...
4,4,24,"[The, jury, said, it, did, find, that, many, o...",The jury said it did find that many of Georgia...


In [12]:
sent_df.sort_values(by=['length'],ascending=False)

,position,length,tokens,sentence
28302,28302,180,"[and, (, C, ), to, finance, ,, for, not, more,...","and ( C ) to finance , for not more than three..."
45083,45083,172,"[He, sucked, in, his, breath, and, kept, quiet...",He sucked in his breath and kept quiet while K...
13174,13174,161,"[``, We, ,, the, Subscribers, ,, do, agree, ,,...","`` We , the Subscribers , do agree , that as s..."
26764,26764,146,"[There, is, ,, of, course, ,, nothing, new, ab...","There is , of course , nothing new about dysto..."
29382,29382,145,"[Inventory, and, evaluate, wildlife, habitat, ...",Inventory and evaluate wildlife habitat resour...
...,...,...,...,...
4823,4823,1,[.],.
33331,33331,1,[Summary],Summary
24598,24598,1,[''],''
48418,48418,1,[''],''


<span style="color:blue">**The longest sentence Brown corpus has has 180 words and shortest is of 1 word, which looks like has only punctuations sometimes**</span>

<a id="patternpos"></a>
<a href="#top">Back to Top</a>
### PATTERN POS Tagger

Apply **Pattern POS Tagger** to all the sentences in Brown corpus for further study

In [13]:
pattern_pos=[]
result_pt_pos=[]
for index in range(0,len(sent_df)):
    #print("index :",index)
    s_p=pos_pattern(sent_df.sentence.iloc[index])
    #print(s_p)
    pattern_pos.append(s_p)
    result_pt_pos.append(compare_pos(brown_goldentag[sent_df.position.iloc[index]],s_p,'pattern_pos',False))

sent_df['pattern_pos_tagger']=result_pt_pos

In [14]:
sent_df[sent_df['pattern_pos_tagger']==True].sort_values(by=['length'],ascending=False)

,position,length,tokens,sentence,pattern_pos_tagger
1862,1862,52,"[Also, Mrs., Berton, Korman, ,, Mrs., Morton, ...","Also Mrs. Berton Korman , Mrs. Morton Rosen , ...",True
27794,27794,40,"[It, embraced, determining, when, to, purchase...",It embraced determining when to purchase and w...,True
28141,28141,32,"[These, three, installment, dates, would, be, ...",These three installment dates would be : Octob...,True
15042,15042,29,"[Here, may, be, found, regular, and, impact, s...","Here may be found regular and impact styrene ,...",True
45184,45184,28,"[I, knew, the, only, way, I, could, beat, you,...",I knew the only way I could beat you was to pl...,True
...,...,...,...,...,...
7196,7196,1,[)],),True
6887,6887,1,[.],.,True
6849,6849,1,[Vacancy],Vacancy,True
27049,27049,1,[)],),True


<span style="color:blue">**Based on the comparisions of Pattern POS tags and the brown corpus tags, 4646 sentences in total had a match out of 57340 sentences. The longest word with a matching POS tag is 52 words long and shortest is 1 word long**</span>

<a id="patternposlong"></a>
<a href="#top">Back to Top</a>

<span style="color:blue">**Longest sentence correctly tagged , when compared to Brown corpus tags, by Pattern POS tagger**</span>

original sentence -- is the input  
golden tag -- is the brown corpus tag  
POS tag -- pattern_pos -- is the tags returned by pattern POS , modified using the dictionary to match the namesin brown corpus

In [15]:
pos_pattern_disp_byindex(1862,brown_goldentag,sent_df,'pattern_pos')

original sentence --- :
Also Mrs. Berton Korman , Mrs. Morton Rosen , Mrs. Jacques Zinman , Mrs. Evelyn Rosen , Mrs. Henry Schultz , Mr. and Mrs. I. S. Kamens , Mrs. Jack Langsdorf , Mrs. Leonard Liss , Mrs. Gordon Blumberg , Mrs. Oscar Bregman , Mrs. Alfred Kershbaum and Mrs. Edward Sabol .
POS tag is correct and matches the golden tag
golden tag ---:
Also/ADV Mrs./NOUN Berton/NOUN Korman/NOUN ,/. Mrs./NOUN Morton/NOUN Rosen/NOUN ,/. Mrs./NOUN Jacques/NOUN Zinman/NOUN ,/. Mrs./NOUN Evelyn/NOUN Rosen/NOUN ,/. Mrs./NOUN Henry/NOUN Schultz/NOUN ,/. Mr./NOUN and/CONJ Mrs./NOUN I./NOUN S./NOUN Kamens/NOUN ,/. Mrs./NOUN Jack/NOUN Langsdorf/NOUN ,/. Mrs./NOUN Leonard/NOUN Liss/NOUN ,/. Mrs./NOUN Gordon/NOUN Blumberg/NOUN ,/. Mrs./NOUN Oscar/NOUN Bregman/NOUN ,/. Mrs./NOUN Alfred/NOUN Kershbaum/NOUN and/CONJ Mrs./NOUN Edward/NOUN Sabol/NOUN ./.
POS tag --- pattern_pos :
Also/ADV Mrs./NOUN Berton/NOUN Korman/NOUN ,/. Mrs./NOUN Morton/NOUN Rosen/NOUN ,/. Mrs./NOUN Jacques/NOUN Zinman/NOUN ,/. M

<a id="patternposshort"></a>
<a href="#top">Back to Top</a>

<span style="color:blue">**Shortest sentence incorrectly tagged, when compared to Brown corpus tags, by Pattern POS tagger**</span>

We have seen sentences with 1 words , often containing punctuations only. And these have been tagged incorrectly for some. Although these may be valid sentences, usually a sentence should have atleast 1 verb and 1 noun , that makes it 2 words per sentence. For our analysis we will consider 2 word sentences as the minimum length of sentence to check the incorrectly tagged sentences

In [16]:

sent_df[(sent_df['pattern_pos_tagger']==False) & (sent_df['length']<10) &(sent_df['length']>1)].sort_values(by=['length'],ascending=True)

,position,length,tokens,sentence,pattern_pos_tagger
27609,27609,2,"[2, .]",2 .,False
36029,36029,2,"[4, .]",4 .,False
35692,35692,2,"[Electric, power]",Electric power,False
51224,51224,2,"[Strange, .]",Strange .,False
3249,3249,2,"[Murphy, honors]",Murphy honors,False
...,...,...,...,...,...
38730,38730,9,"[It, was, still, Good, Friday, ,, after, all, .]","It was still Good Friday , after all .",False
50362,50362,9,"[He, took, a, long, but, carefully, controlled...",He took a long but carefully controlled draught .,False
50361,50361,9,"[He, cleansed, his, mouth, with, a, small, qua...",He cleansed his mouth with a small quantity .,False
38556,38556,9,"[He, waved, his, arm, around, at, the, furnish...",He waved his arm around at the furnishings .,False


In [17]:
pos_pattern_disp_byindex(35692,brown_goldentag,sent_df,'pattern_pos')

original sentence --- :
Electric power
POS tag may not be correct as it does not matches the golden tag
golden tag ---:
Electric/ADJ power/NOUN
POS tag --- pattern_pos :
Electric/NOUN power/NOUN


<span style="color:blue">**Above is the tags of a 2 word sentence that was incorrectly tagged. As per Brown corpus Eating facilities , has a adjective and noun, but our Pattern POS tagger, tags both the words as Noun. 
Brown Corpus tags were created by members of the humanistic world in various academic institutions, including Brown University.**
Comparing other similar 2 word sentences, brown corpus has wither a combination of verb-noun, adj-noun, adv-noun but pattern has identified these as noun-noun.</span>

Let's look  at some of the longer sentences that are incorrectly tagged.

In [18]:
pd.options.display.max_rows = 10
sent_df[(sent_df['pattern_pos_tagger']==False) & (sent_df['length']==5)].sort_values(by=['position'],ascending=True)

,position,length,tokens,sentence,pattern_pos_tagger
187,187,5,"[``, Must, solve, problem, '']",`` Must solve problem '',False
981,981,5,"[--, emphasizes, the, Virgin, birth]",-- emphasizes the Virgin birth,False
1004,1004,5,"[', church, meets, change, ']",' church meets change ',False
1069,1069,5,"[Seeks, ``, improved, fielding, '']",Seeks `` improved fielding '',False
1074,1074,5,"[Duren, ,, Sheldon, on, hill]","Duren , Sheldon on hill",False
...,...,...,...,...,...
57059,57059,5,"[It, wasn't, very, funny, .]",It wasn't very funny .,False
57066,57066,5,"[That, helped, a, lot, .]",That helped a lot .,False
57077,57077,5,"[We, couldn't, help, laughing, .]",We couldn't help laughing .,False
57144,57144,5,"[``, We, don't, '', .]",`` We don't '' .,False


In [19]:
pos_pattern_disp_byindex(57077,brown_goldentag,sent_df,'pattern_pos')

original sentence --- :
We couldn't help laughing .
POS tag may not be correct as it does not matches the golden tag
golden tag ---:
We/PRON couldn't/VERB help/VERB laughing/VERB ./.
POS tag --- pattern_pos :
We/PRON could/VERB n't/ADV help/VERB laughing/VERB ./.


<span style="color:blue">From the incorrectly tagged sentences, picking up a sentence to analyze that has shorthands. And see how it is tagged ?
As we can see above, the difference is due to how pattern tokenizes the word couldn't. Brown corpus identifies it as a single word, "couldn't" but pattern breaks it as "could" and "n't" . Rest of the words have been tagged correctly.  
_Pattern.en module 's tag function, accepts a string and internally calls a tokenize function which returns a list of sentences , with punctuations marks split from words._</span>

In [20]:
pos_pattern("couldn't")

[('could', 'VERB'), ("n't", 'ADV')]

<a id="spacypos"></a>
<a href="#top">Back to Top</a>
### Using spaCy

In [21]:
spacy_pos=[]
result_sp_pos=[]
for index in range(0,len(sent_df)):
    #print("index :",index)
    s_s=pos_spacy(sent_df.sentence.iloc[index])
    #print(s_p)
    spacy_pos.append(s_s)
    result_sp_pos.append(compare_pos(brown_goldentag[sent_df.position.iloc[index]],s_s,'spacy_pos',False))

sent_df['spacy_pos_tagger']=result_sp_pos

<a id="spacyposlong"></a>
<a href="#top">Back to Top</a>

**Longest sentence correctly matched, when compared to Brown corpus tags, returned by spaCy POS tagger**

In [22]:
sent_df[sent_df['spacy_pos_tagger']==True].sort_values(by=['length'],ascending=False)

,position,length,tokens,sentence,pattern_pos_tagger,spacy_pos_tagger
32265,32265,57,"[In, repetitions, of, the, experiment, from, c...",In repetitions of the experiment from couple t...,False,True
33410,33410,51,"[A, borderline, schizophrenic, young, man, tol...",A borderline schizophrenic young man told me t...,False,True
32732,32732,51,"[Of, startling, significance, ,, too, ,, is, t...","Of startling significance , too , is the asser...",False,True
15980,15980,46,"[Extensive, observations, by, physicians, duri...",Extensive observations by physicians during va...,False,True
18083,18083,46,"[The, best, rule, of, thumb, for, detecting, c...",The best rule of thumb for detecting corked wi...,False,True
...,...,...,...,...,...,...
26598,26598,1,[8],8,True,True
27049,27049,1,[)],),True,True
27373,27373,1,[Contact],Contact,True,True
27440,27440,1,[Loans],Loans,True,True


<span style="color:blue">**spaCy POS Tagger returned 8978 tags matching the Brown corpus tags**</span>

**Longest sentence that was correctly matched by spaCy POS Tagger , when compared to Brown corpus, is 57 words long,
shortest being 1 word long**

In [23]:
pos_spacy_disp_byindex(32265,brown_goldentag,sent_df,'spacy_pos')

original sentence --- :
In repetitions of the experiment from couple to couple , the votes of the two persons in a couple probably agree more often than independence would imply , because couples who visit the museum together are more likely to have similar tastes than are a random pair of people drawn from the entire population of visitors .
POS tag is correct and matches the golden tag
golden tag ---:
In/ADP repetitions/NOUN of/ADP the/DET experiment/NOUN from/ADP couple/NOUN to/ADP couple/NOUN ,/. the/DET votes/NOUN of/ADP the/DET two/NUM persons/NOUN in/ADP a/DET couple/NOUN probably/ADV agree/VERB more/ADV often/ADV than/ADP independence/NOUN would/VERB imply/VERB ,/. because/ADP couples/NOUN who/PRON visit/VERB the/DET museum/NOUN together/ADV are/VERB more/ADV likely/ADJ to/PRT have/VERB similar/ADJ tastes/NOUN than/ADP are/VERB a/DET random/ADJ pair/NOUN of/ADP people/NOUN drawn/VERB from/ADP the/DET entire/ADJ population/NOUN of/ADP visitors/NOUN ./.
POS tag --- spacy_pos :
In

<a id="spacyposshort"></a>
<a href="#top">Back to Top</a>

**Shortest sentence correctly matched, when compared to Brown corpus tags, returned by spaCy POS tagger**

In [24]:
pd.options.display.max_rows = 10
sent_df[(sent_df['spacy_pos_tagger']==False) & (sent_df['length']<10) &(sent_df['length']>1)].sort_values(by=['length'],ascending=True)

,position,length,tokens,sentence,pattern_pos_tagger,spacy_pos_tagger
30828,30828,2,"[3, .]",3 .,True,False
34812,34812,2,"[5, .]",5 .,True,False
34816,34816,2,"[6, .]",6 .,True,False
34824,34824,2,"[1, .]",1 .,True,False
34830,34830,2,"[2, .]",2 .,False,False
...,...,...,...,...,...,...
38481,38481,9,"[It, was, left, out, of, him, at, birth, .]",It was left out of him at birth .,False,False
50932,50932,9,"[I've, been, that, far, half, a, dozen, times, .]",I've been that far half a dozen times .,False,False
38503,38503,9,"[Jane, asked, in, her, placid, ,, interested, ...","Jane asked in her placid , interested way .",False,False
50995,50995,9,"[``, Yes, '', ,, Herr, Schaffner, had, said, .]","`` Yes '' , Herr Schaffner had said .",False,False


In [25]:
sent_df[(sent_df['spacy_pos_tagger']==False) & (sent_df['length']==5)].sort_values(by=['position'],ascending=True)

,position,length,tokens,sentence,pattern_pos_tagger,spacy_pos_tagger
187,187,5,"[``, Must, solve, problem, '']",`` Must solve problem '',False,False
676,676,5,"[Formula, is, due, this, week]",Formula is due this week,True,False
962,962,5,"[Oak, Grove, (, special, )]",Oak Grove ( special ),True,False
981,981,5,"[--, emphasizes, the, Virgin, birth]",-- emphasizes the Virgin birth,False,False
1069,1069,5,"[Seeks, ``, improved, fielding, '']",Seeks `` improved fielding '',False,False
...,...,...,...,...,...,...
57066,57066,5,"[That, helped, a, lot, .]",That helped a lot .,False,False
57077,57077,5,"[We, couldn't, help, laughing, .]",We couldn't help laughing .,False,False
57110,57110,5,"[Arlene, became, indispensable, ;, ;]",Arlene became indispensable ; ;,True,False
57144,57144,5,"[``, We, don't, '', .]",`` We don't '' .,False,False


In [13]:
pos_spacy_disp_byindex(35692,brown_goldentag,sent_df,'spacy_pos')

original sentence --- :
Electric power
POS tag is correct and matches the golden tag
golden tag ---:
Electric/ADJ power/NOUN
POS tag --- spacy_pos :
Electric/ADJ power/NOUN


In [26]:
pos_spacy_disp_byindex(57077,brown_goldentag,sent_df,'spacy_pos')

original sentence --- :
We couldn't help laughing .
POS tag may not be correct as it does not matches the golden tag
golden tag ---:
We/PRON couldn't/VERB help/VERB laughing/VERB ./.
POS tag --- spacy_pos :
We/PRON could/AUX n't/ADV help/VERB laughing/VERB ./.


<a id="comppatternvsspacy"></a>
<a href="#top">Back to Top</a>

### Compare pattern POS tagger with spaCy POS tagger 

**Let's look at the sentences we identified from Pattern POS tagger as the longest matching and shorter not matching**

Consider pattern tag as the golden tag now and compare it with the tags returned by spacy

**_Longest correct sentence at index# 1862_**

In [27]:
## Looking at the longest sentence
pos_patt_brown1862=pos_pattern(sent_df.sentence.iloc[1862])
pos_spa_brown1862=pos_spacy(sent_df.sentence.iloc[1862])
compare_pos(pos_patt_brown1862,pos_spa_brown1862,"patternVsspacy", True)

POS tag may not be correct as it does not matches the golden tag
golden tag ---:
Also/ADV Mrs./NOUN Berton/NOUN Korman/NOUN ,/. Mrs./NOUN Morton/NOUN Rosen/NOUN ,/. Mrs./NOUN Jacques/NOUN Zinman/NOUN ,/. Mrs./NOUN Evelyn/NOUN Rosen/NOUN ,/. Mrs./NOUN Henry/NOUN Schultz/NOUN ,/. Mr./NOUN and/CONJ Mrs./NOUN I./NOUN S./NOUN Kamens/NOUN ,/. Mrs./NOUN Jack/NOUN Langsdorf/NOUN ,/. Mrs./NOUN Leonard/NOUN Liss/NOUN ,/. Mrs./NOUN Gordon/NOUN Blumberg/NOUN ,/. Mrs./NOUN Oscar/NOUN Bregman/NOUN ,/. Mrs./NOUN Alfred/NOUN Kershbaum/NOUN and/CONJ Mrs./NOUN Edward/NOUN Sabol/NOUN ./.
POS tag --- patternVsspacy :
Also/ADV Mrs./PROPN Berton/PROPN Korman/PROPN ,/. Mrs./PROPN Morton/PROPN Rosen/PROPN ,/. Mrs./PROPN Jacques/PROPN Zinman/PROPN ,/. Mrs./PROPN Evelyn/PROPN Rosen/PROPN ,/. Mrs./PROPN Henry/PROPN Schultz/PROPN ,/. Mr./PROPN and/CCONJ Mrs./PROPN I./PROPN S./PROPN Kamens/PROPN ,/. Mrs./PROPN Jack/PROPN Langsdorf/PROPN ,/. Mrs./PROPN Leonard/PROPN Liss/PROPN ,/. Mrs./PROPN Gordon/PROPN Blumberg/P

False

In [28]:
print("spacy POS tagger : ", sent_df.spacy_pos_tagger.iloc[1862])

spacy POS tagger :  False


<span style="color:blue">From the above comparison, we can see that **Titles** have been identified as a "NOUN" by Pattern but a "PROPN" by spaCy. This sentence was a match with Brown corpus for pattern and is a mismatch for spaCy.</span>

**_Short incorrect sentence at index# 57077_**

In [29]:
## Looking at the longest sentence
pos_patt_brown57077=pos_pattern(sent_df.sentence.iloc[57077])
pos_spa_brown57077=pos_spacy(sent_df.sentence.iloc[57077])
compare_pos(pos_patt_brown57077,pos_spa_brown57077,"patternVsspacy", True)

POS tag may not be correct as it does not matches the golden tag
golden tag ---:
We/PRON could/VERB n't/ADV help/VERB laughing/VERB ./.
POS tag --- patternVsspacy :
We/PRON could/AUX n't/ADV help/VERB laughing/VERB ./.


False

In [30]:
tag(sent_df.sentence.iloc[57077],tagset=UNIVERSAL)

[('We', 'PR'),
 ('could', 'VB'),
 ("n't", 'RB'),
 ('help', 'VB'),
 ('laughing', 'VB'),
 ('.', '.')]

<span style="color:blue">Auxillery verb is a part of the Universal dataset, but Pattern recognized it as a Verb and spaCy correctly identified it as an auxillery. In this part that is the only difference.
Pattern function ran without any failure for missing corresponding word in the dictionary for "AUX" , meaning it did not identify any Auxillery words.
To avoid mismatches due to combined tags, function is splitting on first hyphen considering first part of a combined tag. Ran the original pattern tag function and it has identified could as just a "verb" and does not include any combined tag.</span>

<span style="color:blue">**Pattern returned POS tags and spaCy returned POS tags , using the universal tagset , produces different results for the 2 sentences compared**</span>

In [31]:
## Looking at the longest sentence
pos_patt_penn_brown1862=pos_pattern_penn(sent_df.sentence.iloc[1862])
pos_spa_penn_brown1862=pos_spacy_penn(sent_df.sentence.iloc[1862])
compare_pos(pos_patt_penn_brown1862,pos_spa_penn_brown1862,"patternVsspacy", True)
print()
print()
## Looking at the longest sentence
pos_patt_penn_brown57077=pos_pattern_penn(sent_df.sentence.iloc[57077])
pos_spa_penn_brown57077=pos_spacy_penn(sent_df.sentence.iloc[57077])
compare_pos(pos_patt_penn_brown57077,pos_spa_penn_brown57077,"patternVsspacy", True)

POS tag is correct and matches the golden tag
golden tag ---:
Also/RB Mrs./NNP Berton/NNP Korman/NNP ,/, Mrs./NNP Morton/NNP Rosen/NNP ,/, Mrs./NNP Jacques/NNP Zinman/NNP ,/, Mrs./NNP Evelyn/NNP Rosen/NNP ,/, Mrs./NNP Henry/NNP Schultz/NNP ,/, Mr./NNP and/CC Mrs./NNP I./NNP S./NNP Kamens/NNP ,/, Mrs./NNP Jack/NNP Langsdorf/NNP ,/, Mrs./NNP Leonard/NNP Liss/NNP ,/, Mrs./NNP Gordon/NNP Blumberg/NNP ,/, Mrs./NNP Oscar/NNP Bregman/NNP ,/, Mrs./NNP Alfred/NNP Kershbaum/NNP and/CC Mrs./NNP Edward/NNP Sabol/NNP ./.
POS tag --- patternVsspacy :
Also/RB Mrs./NNP Berton/NNP Korman/NNP ,/, Mrs./NNP Morton/NNP Rosen/NNP ,/, Mrs./NNP Jacques/NNP Zinman/NNP ,/, Mrs./NNP Evelyn/NNP Rosen/NNP ,/, Mrs./NNP Henry/NNP Schultz/NNP ,/, Mr./NNP and/CC Mrs./NNP I./NNP S./NNP Kamens/NNP ,/, Mrs./NNP Jack/NNP Langsdorf/NNP ,/, Mrs./NNP Leonard/NNP Liss/NNP ,/, Mrs./NNP Gordon/NNP Blumberg/NNP ,/, Mrs./NNP Oscar/NNP Bregman/NNP ,/, Mrs./NNP Alfred/NNP Kershbaum/NNP and/CC Mrs./NNP Edward/NNP Sabol/NNP ./.


POS

True

<span style="color:blue">**Pattern returned POS tags and spaCy returned POS tags , using the Penn-Tag tagset , produces same results for the 2 sentences compared**</span>

<a id="exppatternvsspacy"></a>
<a href="#top">Back to Top</a>

### Explain any differences between Pattern POS tagger and spaCy POS tagger

For starters :
1. Pattern universal POS tagger returned 4646 matching tags while spaCy universal POS tagger returned 8978 matching tags when compared to Brown corpus  
2. Pattern uses Universal tagset while spaCy uses Google's Universal tagset which has couple more additions  
3. Pattern was faster to process the 57340 sentences than spaCy
4. For the sentences compared, there were differences in the universal tags returned by pattern and spacy but their tags matched when using the Penn-Tag tagset. Which means there could be more matches between pattern and spacy using the Penn-Tag tagset
5. Pattern did not identify Auxillery verbs differently than Verbs , although AUX is part of the universal tagset.
6. spaCy identified Titles e.g. 'Mrs' as a ProperNoun while pattern identified it as a Noun as the humanists for the Brown Corpus did

<a id="news"></a>
<a href="#top">Back to Top</a>
### News article

In [32]:
news="Boosted by a weaker dollar and fresh enthusiasm for cryptocurrencies, bitcoin surged past $10,000 for the first time in a year."

<a id="nenewsmanualws"></a>
<a href="#top">Back to Top</a>

Manually create POS tags using the Penn-Tag tagset. Trying my best here..

In [33]:
pos_manual=[('Boosted', 'VB'),
 ('by', 'IN'),
 ('a', 'DT'),
 ('weaker', 'JJR'),
 ('dollar', 'NN'),
 ('and', 'CC'),
 ('fresh', 'JJ'),
 ('enthusiasm', 'NN'),
 ('for', 'IN'),
 ('cryptocurrencies', 'NN'),
 (',', ','),
 ('bitcoin', 'NN'),
 ('surged', 'VB'),
 ('past', 'IN'),
 ('$', '$'),
 ('10,000', 'CD'),
 ('for', 'IN'),
 ('the', 'DT'),
 ('first', 'CD'),
 ('time', 'NN'),
 ('in', 'IN'),
 ('a', 'DT'),
 ('year', 'NN'),
 ('.', '.')]

<a id="newscomppatternvsspacy"></a>
<a href="#top">Back to Top</a>

### Compare Manual POS tags vs Pattern POS tags vs SpaCy POS tags using Penn-Tag tagset

In [34]:
pos_patt=pos_pattern_penn(news)
pos_spac=pos_spacy_penn(news)
news_pos_df=pd.DataFrame({"manual":pos_manual,
                         "pattern":pos_patt,
                         "spacy":pos_spac})

#### Graphical view of the SpaCy POS Tagger

In [35]:
pos_spac_viz=sp_nlp(news)
sentence_spans=list(pos_spac_viz.sents)
displacy.serve(sentence_spans, style="dep", minify=True, options={"compact":True})

/Users/tanviarora/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


**Compare 2 tags at a time**

In [47]:
compare_pos(pos_manual,pos_patt,"manualVspattern", True)

POS tag may not be correct as it does not matches the golden tag
golden tag ---:
Boosted/VB by/IN a/DT weaker/JJR dollar/NN and/CC fresh/JJ enthusiasm/NN for/IN cryptocurrencies/NN ,/, bitcoin/NN surged/VB past/IN $/$ 10,000/CD for/IN the/DT first/CD time/NN in/IN a/DT year/NN ./.
POS tag --- manualVspattern :
Boosted/VBD by/IN a/DT weaker/JJR dollar/NN and/CC fresh/JJ enthusiasm/NN for/IN cryptocurrencies/NNS ,/, bitcoin/NN surged/VBD past/RB $/$ 10,000/CD for/IN the/DT first/JJ time/NN in/IN a/DT year/NN ./.


False

In [48]:
compare_pos(pos_manual,pos_spac,"manualVsspacy", True)

POS tag may not be correct as it does not matches the golden tag
golden tag ---:
Boosted/VB by/IN a/DT weaker/JJR dollar/NN and/CC fresh/JJ enthusiasm/NN for/IN cryptocurrencies/NN ,/, bitcoin/NN surged/VB past/IN $/$ 10,000/CD for/IN the/DT first/CD time/NN in/IN a/DT year/NN ./.
POS tag --- manualVsspacy :
Boosted/VBN by/IN a/DT weaker/JJR dollar/NN and/CC fresh/JJ enthusiasm/NN for/IN cryptocurrencies/NNS ,/, bitcoin/NN surged/VBD past/IN $/$ 10,000/CD for/IN the/DT first/JJ time/NN in/IN a/DT year/NN ./.


False

In [49]:
compare_pos(pos_patt,pos_spac,"patternVsspacy", True)

POS tag may not be correct as it does not matches the golden tag
golden tag ---:
Boosted/VBD by/IN a/DT weaker/JJR dollar/NN and/CC fresh/JJ enthusiasm/NN for/IN cryptocurrencies/NNS ,/, bitcoin/NN surged/VBD past/RB $/$ 10,000/CD for/IN the/DT first/JJ time/NN in/IN a/DT year/NN ./.
POS tag --- patternVsspacy :
Boosted/VBN by/IN a/DT weaker/JJR dollar/NN and/CC fresh/JJ enthusiasm/NN for/IN cryptocurrencies/NNS ,/, bitcoin/NN surged/VBD past/IN $/$ 10,000/CD for/IN the/DT first/JJ time/NN in/IN a/DT year/NN ./.


False

In [50]:
pd.options.display.max_rows = 25
news_pos_df

,manual,pattern,spacy
0,"(Boosted, VB)","(Boosted, VBD)","(Boosted, VBN)"
1,"(by, IN)","(by, IN)","(by, IN)"
2,"(a, DT)","(a, DT)","(a, DT)"
3,"(weaker, JJR)","(weaker, JJR)","(weaker, JJR)"
4,"(dollar, NN)","(dollar, NN)","(dollar, NN)"
5,"(and, CC)","(and, CC)","(and, CC)"
6,"(fresh, JJ)","(fresh, JJ)","(fresh, JJ)"
7,"(enthusiasm, NN)","(enthusiasm, NN)","(enthusiasm, NN)"
8,"(for, IN)","(for, IN)","(for, IN)"
9,"(cryptocurrencies, NN)","(cryptocurrencies, NNS)","(cryptocurrencies, NNS)"


<a id="newsexppatternvsspacy"></a>
<a href="#top">Back to Top</a>

### Explain any differences between Manual POS tagger ,Pattern POS tagger and spaCy POS tagger

From the compare function we know that the 3 tags are different. Manually analyzing them, we see that there are few differences. Let's discuss them below :

1. Boosted : I have plainly mentioned it as a 'VB'. Penn-Tag tagset offers more varieties to verbs and probably I used it wrong here. But pattern recognizes it as a 'VBD' i.e. "verb,past tense" and spaCy recognized it as 'VBN' , i.e. "verb.past participle" . 
2. cryptocurrencies : Again I have just considered it as a Noun(NN), but applying the varieties offered by Penn-Tag tagset, both pattern and spacy match it as 'NNS' which is "Noun Plural"
3. surged : 'VB' i.e Verb marked by me is 'VBD' i.e. "verb,past tense" by both pattern and spaCy
4. first : Incorrectly tagged by me as a 'CD' i.e. "Cardinal" , is clearly an adjective as correctly marked by both pattern and spacy as it describes the noun 'time'

<span style="color:blue">**Overall looks like we have 4 differences , considering the tagset used, all 4 incorrectly tagged by me.
Pattern and spaCy on the other hand have only 1 difference for the word 'Boosted' , sounds like pattern could be a winner here with verb,past tense**</span>

### What worked and What did not ??

1. First attempt was made to compare Brown corpus tags in default/PennTag tagset with Pattern's default PennTag tagset. This returned the longest sentence lenght of matching tags as only 9. Further analysis pointed out that Pattern's PennTag tagger was considering a double quote(") as 2 single quotes('). This is definitely something to look out for. Tried some logic for ignoring punctuation characters in this case but it requires more work. Not presented in this book
2. PenTagg tagset returned by brown corpus, has a few combined tags. These combined tags are used differently. Tried splitting the tags on hyphen "-" that joins a combined tag, but still same result, longest matching sentence returned was of 9 words.
3. Finally used UNIVERSAL tagset to compare the tags automatically. This yielded better results , but required use of dictionary as the outputs in pattern and spacy were slightly different.


<a id="addtagers"></a>
<a href="#top">Back to Top</a>
### Additional POS taggers ??

#### NLTK ClassiferBasedPOSTagger

In [15]:
from nltk.tag.sequential import ClassifierBasedPOSTagger
from nltk.corpus import treebank,brown

In [16]:
train_data=brown.tagged_sents(tagset='universal')
cbpt_tagging = ClassifierBasedPOSTagger(train = train_data)

def pos_cbpt_uni(tokens):
    return cbpt_tagging.tag(tokens)

## returns spaCy POS tags for the sentence at a given index along with its comparision with golden tagset
def pos_cbpt_disp_byindex(ind,golden_tag_lst,input_df,pos_tag_name):
    cbpt_pos_ind=pos_cbpt_uni(input_df.tokens.iloc[ind])
    brown_golden_ind=golden_tag_lst[ind]
    print("original sentence --- :")
    print(input_df.sentence.iloc[ind])
    compare_pos(brown_golden_ind,cbpt_pos_ind,pos_tag_name,True)

In [53]:
cbpt_pos=[]
result_cb_pos=[]
for index in range(0,len(sent_df)):
    #print("index :",index)
    cb_s=pos_cbpt_uni(sent_df.tokens.iloc[index])
    #print(s_p)
    cbpt_pos.append(cb_s)
    result_cb_pos.append(compare_pos(brown_goldentag[sent_df.position.iloc[index]],cb_s,'cbpt_pos',False))

sent_df['cbpt_pos_tagger']=result_cb_pos

In [54]:
sent_df[sent_df['cbpt_pos_tagger']==True].sort_values(by=['length'],ascending=False)

,position,length,tokens,sentence,pattern_pos_tagger,spacy_pos_tagger,cbpt_pos_tagger
33619,33619,82,"[The, great, majority, of, present-day, lingui...",The great majority of present-day linguists fa...,False,False,True
20159,20159,80,"[Our, leadership, in, a, wide, economic, boyco...",Our leadership in a wide economic boycott of S...,False,False,True
26626,26626,78,"[But, his, rancor, did, not, cease, ,, and, pr...","But his rancor did not cease , and presently ,...",False,False,True
29684,29684,77,"[For, this, reason, ,, the, more, uncertain, s...","For this reason , the more uncertain skywave s...",False,False,True
35766,35766,75,"[To, derive, Utopian, communism, from, the, Je...",To derive Utopian communism from the Jerusalem...,False,False,True
32220,32220,75,"[For, an, experiment, to, qualify, as, a, bino...",For an experiment to qualify as a binomial exp...,False,False,True
6370,6370,74,"[The, Illinois, Commission, for, Handicapped, ...",The Illinois Commission for Handicapped Childr...,False,False,True
53056,53056,73,"[She, glanced, at, the, man, nodding, beside, ...","She glanced at the man nodding beside her , a ...",False,False,True
13117,13117,73,"[The, drill, press, consists, of, a, vertical,...",The drill press consists of a vertical shaft (...,False,False,True
16016,16016,72,"[Such, understanding, helps, to, explain, why,...",Such understanding helps to explain why one ma...,False,False,True


<span style="color:blue">**ClassiferBasedPOSTagger trained on brown corpus itself, returned 31360 matched tags**</span>

**The longest matching sentence , when compared to brown corpus tags, has a length of 82 words and shortest not matching sentence is 1 word long**
Definitely this is higher accuracy as compared to the other taggers, but we should not forget we trained this Tagger on brown corpus tagged words itself. 

### Check output for the same sentences from Pattern 

In [18]:
pos_cbpt_disp_byindex(32265,brown_goldentag,sent_df,'ClassifierBased_pos')

original sentence --- :
In repetitions of the experiment from couple to couple , the votes of the two persons in a couple probably agree more often than independence would imply , because couples who visit the museum together are more likely to have similar tastes than are a random pair of people drawn from the entire population of visitors .
POS tag may not be correct as it does not matches the golden tag
golden tag ---:
In/ADP repetitions/NOUN of/ADP the/DET experiment/NOUN from/ADP couple/NOUN to/ADP couple/NOUN ,/. the/DET votes/NOUN of/ADP the/DET two/NUM persons/NOUN in/ADP a/DET couple/NOUN probably/ADV agree/VERB more/ADV often/ADV than/ADP independence/NOUN would/VERB imply/VERB ,/. because/ADP couples/NOUN who/PRON visit/VERB the/DET museum/NOUN together/ADV are/VERB more/ADV likely/ADJ to/PRT have/VERB similar/ADJ tastes/NOUN than/ADP are/VERB a/DET random/ADJ pair/NOUN of/ADP people/NOUN drawn/VERB from/ADP the/DET entire/ADJ population/NOUN of/ADP visitors/NOUN ./.
POS tag

<span style="color:blue">This was the longest sentence correctly tagged , when compared with Brown Corpus, by pattern and we checked output for the same with spaCy POS Tagger as well. Both the initial taggers tagged it correctly but our ClassifierBased Model failed to do so.</span>  
<span style="color:blue">The only error i see is for couple 'to' couple where as per Brown Corpus 'to' is AdPosition but as per our model it is a particle.</span>

In [19]:
pos_cbpt_disp_byindex(57077,brown_goldentag,sent_df,'ClassifierBased_pos')

original sentence --- :
We couldn't help laughing .
POS tag is correct and matches the golden tag
golden tag ---:
We/PRON couldn't/VERB help/VERB laughing/VERB ./.
POS tag --- ClassifierBased_pos :
We/PRON couldn't/VERB help/VERB laughing/VERB ./.


In [20]:
pos_cbpt_disp_byindex(35692,brown_goldentag,sent_df,'ClassifierBased_pos')

original sentence --- :
Electric power
POS tag is correct and matches the golden tag
golden tag ---:
Electric/ADJ power/NOUN
POS tag --- ClassifierBased_pos :
Electric/ADJ power/NOUN


<span style="color:blue">Both the above short sentences , where pattern failed to match correctly, our Classifierbased POS Tagger matched it correctly. Also to notice it did not break the word "couldn't" into 2 words like pattern and spacy. Infact it kept it as 1 word like the Brown Corpus</span>